# Resume Base Training (Notebook Workflow)

This notebook clones the DiffiT LoRA repository, installs dependencies, and resumes base model training from an existing checkpoint. Update the configuration cell before launching the training step.


## 1. Clone the Repository

Run this cell once per environment. If the repository already exists, it will skip cloning and just set the working directory.


In [ ]:
import os
import sys
from pathlib import Path

REPO_URL = "https://github.com/drhspeco/diffit-lora.git"
TARGET_DIR = Path("diffit-lora")

if (Path.cwd() / ".git").exists():
    project_root = Path.cwd()
    print(f"Repository already available at {project_root}")
else:
    if TARGET_DIR.exists():
        project_root = TARGET_DIR.resolve()
        print(f"Repository already present at {project_root}")
    else:
        print(f"Cloning {REPO_URL} ...")
        !git clone {REPO_URL}
        project_root = TARGET_DIR.resolve()
    os.chdir(project_root)
    print(f"Working directory set to {Path.cwd()}")

if not (Path.cwd() / ".git").exists():
    raise RuntimeError("Could not locate the diffit-lora repository. Update TARGET_DIR or run the cell again from the correct location.")

sys.path.insert(0, str(Path.cwd()))
print("sys.path updated for local imports.")


## 2. (Optional) Mount Google Drive

Run this cell when working in Google Colab so checkpoints and configs stored in Drive are available.


In [ ]:
try:
    from google.colab import drive  # type: ignore
    drive.mount('/content/drive')
    print("Google Drive mounted.")
except ImportError:
    print("Google Colab not detected; skipping Drive mount.")


## 3. Install Dependencies

Make sure the required Python packages are installed. Comment out packages you already have.


In [ ]:
%pip install -q -r requirements.txt


## 4. Configure Paths and Hyperparameters

Update the variables below to point to your checkpoints, config files, and desired overrides before starting the training job.


In [ ]:
from pathlib import Path

# Attempt to use Google Drive copy when available; fall back to local repo directory.
default_drive_root = Path("/content/drive/MyDrive/diffit-lora")
PROJECT_ROOT = default_drive_root if default_drive_root.exists() else Path.cwd()

# Paths to important assets
CHECKPOINT_TO_RESUME = PROJECT_ROOT / "checkpoints" / "base" / "base_model_last.ckpt"
TRAINING_CONFIG_PATH = PROJECT_ROOT / "configs" / "training" / "base_training.yaml"
MODEL_CONFIG_PATH = PROJECT_ROOT / "configs" / "models" / "unet_config.yaml"
DATA_CONFIG_PATH = None  # Optional explicit path; leave as None to resolve from DATASET_NAME

# Training overrides (set to None to keep YAML values)
DATASET_NAME = None          # Examples: "CIFAR", "CIFAR100", "imagenette"
NUM_EPOCHS = None            # e.g., 5
LEARNING_RATE = None         # e.g., 5e-4
BATCH_SIZE = None            # If provided, overrides data config batch size
ACCELERATOR = None           # e.g., "gpu", "cpu", "mps"
DEVICES = None               # e.g., 1 or "auto"
PRECISION = None             # e.g., "16-mixed"
GRADIENT_CLIP_VAL = None
LOG_EVERY_N_STEPS = None

# Model overrides (defaults defined in configs/models/unet_config.yaml)
MODEL_OVERRIDES = {
    "d_model": None,
    "num_heads": None,
    "dropout": None,
    "d_ff": None,
    "img_size": None,
    "denoising_steps": None,
    "L1": None,
    "L2": None,
    "L3": None,
    "L4": None,
}

# Output management
OUTPUT_DIR = PROJECT_ROOT / "checkpoints" / "base_training"
EXPERIMENT_NAME = "diffit_base_resumed"
KEEP_LAST_N = 3
SAVE_EVERY_N_EPOCHS = 1

# Execution toggle
RUN_TRAINING = False  # Switch to True when you are ready to launch training.



## 5. Resume Training Helper

The function below mirrors `resume_base_training.py` but exposes configuration overrides for notebook use.


In [ ]:
from pathlib import Path

import yaml
import torch
import pytorch_lightning as pl
from pytorch_lightning.callbacks import LearningRateMonitor

from diffit.training.base_checkpoint_callbacks import BaseModelCheckpointCallback
from diffit.training.data import DiffiTDataModule
from diffit.models.unet import UShapedNetwork


def _resolve_data_config_path(dataset_name: str, project_root: Path) -> Path:
    dataset_key = dataset_name.upper()
    data_dir = project_root / "configs" / "data"

    if dataset_key == "CIFAR":
        candidate = data_dir / "cifar10.yaml"
    elif dataset_key == "CIFAR100":
        candidate = data_dir / "cifar100.yaml"
    else:
        candidate = data_dir / f"{dataset_name.lower()}.yaml"

    if not candidate.exists():
        raise FileNotFoundError(f"Could not find data config for dataset '{dataset_name}' at {candidate}.")

    return candidate


def _load_model_config(model_config_path: Path, model_overrides: dict | None) -> tuple[dict, torch.device]:
    with open(model_config_path, "r") as f:
        model_yaml = yaml.safe_load(f) or {}

    model_cfg = dict(model_yaml.get("model", {}))
    if model_overrides:
        for key, value in model_overrides.items():
            if value is not None:
                model_cfg[key] = value

    device_pref = model_overrides.get("device") if model_overrides else None
    if device_pref is None:
        device_pref = model_yaml.get("device", "auto")

    if device_pref == "auto":
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    else:
        device = torch.device(device_pref)

    return model_cfg, device


def resume_base_training_notebook(
    checkpoint_path: Path,
    training_config_path: Path,
    output_dir: Path,
    experiment_name: str = "diffit_base_resumed",
    dataset_name: str | None = None,
    data_config_path: Path | None = None,
    num_epochs: int | None = None,
    learning_rate: float | None = None,
    batch_size: int | None = None,
    accelerator: str | None = None,
    devices: str | int | list[int] | None = None,
    precision: str | None = None,
    gradient_clip_val: float | None = None,
    log_every_n_steps: int | None = None,
    model_config_path: Path | None = None,
    model_overrides: dict | None = None,
    keep_last_n: int = 3,
    save_every_n_epochs: int = 1,
    run_training: bool = False,
) -> dict:
    """Resume base training from a checkpoint with notebook-friendly overrides."""

    checkpoint_path = checkpoint_path.expanduser().resolve()
    training_config_path = training_config_path.expanduser().resolve()
    output_dir = output_dir.expanduser().resolve()

    if not checkpoint_path.exists():
        raise FileNotFoundError(f"Checkpoint not found at {checkpoint_path}")
    if not training_config_path.exists():
        raise FileNotFoundError(f"Training config not found at {training_config_path}")

    with open(training_config_path, "r") as f:
        training_yaml = yaml.safe_load(f) or {}

    training_cfg = dict(training_yaml.get("training", {}))

    if dataset_name is not None:
        training_cfg["dataset"] = dataset_name
    if num_epochs is not None:
        training_cfg["num_epochs"] = num_epochs
    if learning_rate is not None:
        training_cfg["learning_rate"] = learning_rate
    if accelerator is not None:
        training_cfg["accelerator"] = accelerator
    if devices is not None:
        training_cfg["devices"] = devices
    if precision is not None:
        training_cfg["precision"] = precision
    if gradient_clip_val is not None:
        training_cfg["gradient_clip_val"] = gradient_clip_val
    if log_every_n_steps is not None:
        training_cfg["log_every_n_steps"] = log_every_n_steps

    resolved_dataset = training_cfg.get("dataset", "CIFAR")
    if data_config_path is None:
        parents = training_config_path.parents
        project_root = parents[2] if len(parents) >= 3 else parents[-1]
        data_config_path = _resolve_data_config_path(resolved_dataset, project_root)

    with open(data_config_path, "r") as f:
        data_config = yaml.safe_load(f) or {}

    if batch_size is not None:
        data_section = data_config.setdefault("data", {})
        data_section["batch_size_train"] = batch_size
        if "batch_size_test" in data_section and batch_size < data_section["batch_size_test"]:
            print("⚠️ batch_size override is smaller than batch_size_test; adjust test batch size if needed.")

    if model_config_path is None:
        raise ValueError("model_config_path must be provided.")

    model_cfg, device = _load_model_config(model_config_path.expanduser().resolve(), model_overrides or {})
    model_cfg.setdefault("learning_rate", training_cfg.get("learning_rate", 1e-3))
    model_cfg["learning_rate"] = training_cfg.get("learning_rate", model_cfg.get("learning_rate", 1e-3))

    output_dir.mkdir(parents=True, exist_ok=True)

    checkpoint_callback = BaseModelCheckpointCallback(
        base_dir=str(output_dir),
        run_number=None,
        experiment_name=experiment_name,
        monitor="train_loss",
        mode="min",
        save_every_n_epochs=save_every_n_epochs,
        keep_last_n=keep_last_n,
        verbose=True,
    )

    lr_monitor = LearningRateMonitor(logging_interval="step")
    callbacks = [checkpoint_callback, lr_monitor]

    data_module = DiffiTDataModule(data_config)

    trainer = pl.Trainer(
        max_epochs=training_cfg.get("num_epochs", 3),
        callbacks=callbacks,
        accelerator=training_cfg.get("accelerator", "auto"),
        devices=training_cfg.get("devices", "auto"),
        precision=training_cfg.get("precision", "32-true"),
        gradient_clip_val=training_cfg.get("gradient_clip_val", 1.0),
        log_every_n_steps=training_cfg.get("log_every_n_steps", 25),
        default_root_dir=str(checkpoint_callback.logs_dir),
    )

    model = UShapedNetwork(
        learning_rate=model_cfg["learning_rate"],
        d_model=model_cfg["d_model"],
        num_heads=model_cfg["num_heads"],
        dropout=model_cfg["dropout"],
        d_ff=model_cfg["d_ff"],
        img_size=model_cfg["img_size"],
        device=device,
        denoising_steps=model_cfg["denoising_steps"],
        L1=model_cfg.get("L1", 2),
        L2=model_cfg.get("L2", 2),
        L3=model_cfg.get("L3", 2),
        L4=model_cfg.get("L4", 2),
    )

    print("\n🔄 Resuming Base Model Training")
    print("=" * 60)
    print(f"Checkpoint: {checkpoint_path}")
    print(f"Dataset config: {data_config_path}")
    print(f"Training config: {training_config_path}")
    print(f"Output directory: {checkpoint_callback.run_dir}")
    print(f"Device: {device}")

    checkpoint = torch.load(checkpoint_path, map_location="cpu")
    starting_epoch = checkpoint.get("epoch", 0)
    checkpoint_callback.set_starting_epoch(starting_epoch)

    checkpoint_fixed = False
    if "pytorch-lightning_version" not in checkpoint:
        checkpoint["pytorch-lightning_version"] = pl.__version__
        checkpoint_fixed = True
    if "optimizer_states" not in checkpoint:
        checkpoint["optimizer_states"] = []
        checkpoint["lr_schedulers"] = []
        checkpoint["optimizer_state_dict"] = None
        checkpoint["scheduler_state_dict"] = None
        checkpoint_fixed = True
        print("🔧 Added missing optimizer/scheduler states (weights-only checkpoint)")

    checkpoint_to_use = checkpoint_path
    if checkpoint_fixed:
        checkpoint_to_use = output_dir / "temp_fixed_checkpoint.ckpt"
        torch.save(checkpoint, checkpoint_to_use)
        print(f"🔧 Fixed checkpoint compatibility - saved to: {checkpoint_to_use}")

    print(f"📊 Continuing from epoch {starting_epoch + 1}")

    if run_training:
        trainer.fit(model, datamodule=data_module, ckpt_path=str(checkpoint_to_use))
        print("Training complete.")
    else:
        print("Dry run complete. Set run_training=True to start training.")

    if checkpoint_fixed and checkpoint_to_use.exists():
        checkpoint_to_use.unlink()
        print("🧹 Cleaned up temporary checkpoint file.")

    return {
        "trainer": trainer,
        "model": model,
        "data_module": data_module,
        "checkpoint_callback": checkpoint_callback,
        "training_config": training_cfg,
        "data_config_path": data_config_path,
    }



## 6. Launch (or Dry Run) Training

Execute the cell below after reviewing the configuration. Set `RUN_TRAINING = True` in the configuration cell to actually start training.


In [ ]:
from IPython.display import display

results = resume_base_training_notebook(
    checkpoint_path=CHECKPOINT_TO_RESUME,
    training_config_path=TRAINING_CONFIG_PATH,
    output_dir=OUTPUT_DIR,
    experiment_name=EXPERIMENT_NAME,
    dataset_name=DATASET_NAME,
    data_config_path=DATA_CONFIG_PATH,
    num_epochs=NUM_EPOCHS,
    learning_rate=LEARNING_RATE,
    batch_size=BATCH_SIZE,
    accelerator=ACCELERATOR,
    devices=DEVICES,
    precision=PRECISION,
    gradient_clip_val=GRADIENT_CLIP_VAL,
    log_every_n_steps=LOG_EVERY_N_STEPS,
    model_config_path=MODEL_CONFIG_PATH,
    model_overrides=MODEL_OVERRIDES,
    keep_last_n=KEEP_LAST_N,
    save_every_n_epochs=SAVE_EVERY_N_EPOCHS,
    run_training=RUN_TRAINING,
)

display({
    "next_checkpoint_dir": str(results["checkpoint_callback"].checkpoint_dir),
    "log_dir": str(results["checkpoint_callback"].logs_dir),
    "resolved_dataset_config": str(results["data_config_path"]),
    "current_training_config": results["training_config"],
})


## 7. Next Steps

- Inspect the returned training configuration to confirm overrides.
- Adjust dataset or model overrides as needed, then set `RUN_TRAINING = True` and re-run the launch cell.
- Monitor checkpoint and log folders reported in the output for progress.
